In [1]:
from NeuralNet import NeuralNet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit
from sklearn.datasets import fetch_openml

In [13]:
NN = NeuralNet(depth=3, learning_rate=0.1, s_in=784, s_out=10)

In [14]:
X, y = fetch_openml("mnist_784", return_X_y=True)

In [15]:
y = np.array([[0 if i != x else 1 for i in range(10)] for x in y])

In [ ]:
loss = NN.train(X, y, 1000)

/Users/mdeville/Projects/Algo/multilayer_perceptron/NeuralNet.py:58: RuntimeWarning: divide by zero encountered in log
  return (- y.T.dot(np.log(h)) - (1 - y).T.dot(np.log(1 - h))) / m


In [ ]:
plt.plot( )